<h2><font face=verdona color=yellow><center>Web Extraction Pattern</center></font></h3>
<h4><font face=verdona >
Structure of Extracting Data from websites<br>
⦿ Searching Reputed Used Car Selling Organization Website - Website1_HomeURL, Website2_HomeURL, ...., WebsiteN_HomeURL<br>
⦿ Searching all Cities each organization operates in and fetching links filtering only that city - Website1 : City1_URL,...,CityN_URL<br>
⦿ Fetching links of all cars sold in each city - City1 : Car1_URL, Car2_URL, Car3_URL,........,CarN_URL<br>
⦿ Extracting only required data from each Car link.<br>
</font></h4>
<pre>
                       Website1                             .....                         WebsiteN
                          |                                                                  |
                 -------------------                                                ------------------------
                 |        |         |                                               |                      | 
              City1     City2     City3                                          City1     .....         CityN
                |         |         |                                              |                       |
    -------------      --------     -----------------                  -------------                   --------------
    |     |     |      |      |         |     |     |                  |           |                   |            |                       
  Car1  Car2  Car3    Car1   Car2      Car1  Car2  Car3              Car1  ....  CarN                 Car1  ....  CarN
    ↓     ↓    ↓       ↓      ↓         ↓      ↓    ↓                  ↓          ↓                     ↓           ↓                                     
      Extracted        Extracted           Extracted                     Extracted                        Extracted                                     
        Data             Data                 Data                         Data                              Data                                       


<h4><font face=verdona>DATA CREDITS - Cars24, Carwaale

<H2><font face=georgia color=cyan><center>Importing Modules

In [1]:
import requests
from bs4 import BeautifulSoup
import time 
import numpy as np
import ast
import os
import re
import pandas as pd
from pathlib import Path    

In [2]:
def fetch_web_homepage(mainpage_link, output_directory,
                   headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
):
    try:
        os.makedirs(output_directory, exist_ok=True)
        available_cities = requests.get(mainpage_link, headers=headers)
        available_cities.raise_for_status() 
    except requests.exceptions.RequestException as e:
        print(f"Error fetching main page {mainpage_link}: {e}")
        return {} 

    soup = BeautifulSoup(available_cities.content, 'lxml')
    with open(os.path.join(output_directory, 'main_cities.txt'), 'w', encoding='utf-8')as f:
        f.write(soup.prettify())

In [3]:
mainpage_cars24 = 'https://www.cars24.com/buy-used-cars/'
mainpage_carwale = 'https://www.carwale.com/used/'
mainpage_cardekho = 'https://www.cardekho.com/used-cars/'
mainpage_zigwheels = 'https://www.zigwheels.com/used-car'
mainpage_cartrade = 'https://www.cartrade.com/second-hand/'

websites = {'cars24' : mainpage_cars24, 'carwale' : mainpage_carwale, 'cardekho' : mainpage_cardekho, 
            'zigwheels': mainpage_zigwheels, 'cartrade': mainpage_cartrade}

In [4]:
# for organization, website_URL in websites.items():
#     fetch_web_homepage(mainpage_link=website_URL, output_directory='organized_cars_structure/'+organization+'_data')

In [5]:
def extract_citywise_links(root_directory, required_links_extractor, city_extractor, html_tag='a'):
    with open(os.path.join(root_directory, 'main_cities.txt'), 'r', encoding='utf-8')as f:
        content = f.read()

    soup = BeautifulSoup(content, 'lxml')
    href_values = [a.get('href') for a in soup.select(html_tag, href=True)]
    city_links = [required_links_extractor(link) for link in href_values]
    cities = [city_extractor(link) for link in city_links]
    citywise_links = dict()

    for city, link in zip(cities, city_links):
        if city.strip() != '':
            citywise_links[city] = link
    return citywise_links

In [6]:
extract_city_links_cars24 = lambda x: (x if 'buy-used-cars-' in x else '')
city_extractor_cars24 = lambda x: x[len('https://www.cars24.com/buy-used-cars-'):-1]

cars24_citylinks = extract_citywise_links(
    root_directory='organized_cars_structure/cars24_data/', required_links_extractor=extract_city_links_cars24, 
    city_extractor=city_extractor_cars24, html_tag='a.sc-kGqRax.bjSmXr')

print('Cars24 Available cities and their links \n','-'*30)
cars24_citylinks

Cars24 Available cities and their links 
 ------------------------------


{'delhi-ncr': 'https://www.cars24.com/buy-used-cars-delhi-ncr/',
 'bangalore': 'https://www.cars24.com/buy-used-cars-bangalore/',
 'chennai': 'https://www.cars24.com/buy-used-cars-chennai/',
 'hyderabad': 'https://www.cars24.com/buy-used-cars-hyderabad/',
 'mumbai': 'https://www.cars24.com/buy-used-cars-mumbai/',
 'ahmedabad': 'https://www.cars24.com/buy-used-cars-ahmedabad/',
 'pune': 'https://www.cars24.com/buy-used-cars-pune/',
 'new-delhi': 'https://www.cars24.com/buy-used-cars-new-delhi/',
 'noida': 'https://www.cars24.com/buy-used-cars-noida/',
 'kochi': 'https://www.cars24.com/buy-used-cars-kochi/',
 'ghaziabad': 'https://www.cars24.com/buy-used-cars-ghaziabad/',
 'gurgaon': 'https://www.cars24.com/buy-used-cars-gurgaon/',
 'kolkata': 'https://www.cars24.com/buy-used-cars-kolkata/',
 'lucknow': 'https://www.cars24.com/buy-used-cars-lucknow/',
 'chandigarh-tricity': 'https://www.cars24.com/buy-used-cars-chandigarh-tricity/'}

In [7]:
extract_city_links_carwale = lambda href_val : 'https://www.carwale.com' + href_val
city_extractor_carwale = lambda href_val: href_val[len('https://www.carwale.com/used/'):-1]

carwale_citylinks = extract_citywise_links(
    root_directory='organized_cars_structure/carwale_data',
    required_links_extractor=extract_city_links_carwale,
    city_extractor=city_extractor_carwale,
    html_tag='a.o-aw.o-aE.o-co.o-ml.o-lT.o-mo.o-mO.o-bS.EW3D5f.o-kY.o-f')

print('Carwale Available cities and their links \n','-'*30)
carwale_citylinks

Carwale Available cities and their links 
 ------------------------------


{'mumbai': 'https://www.carwale.com/used/mumbai/',
 'delhi': 'https://www.carwale.com/used/delhi/',
 'bangalore': 'https://www.carwale.com/used/bangalore/',
 'chennai': 'https://www.carwale.com/used/chennai/',
 'hyderabad': 'https://www.carwale.com/used/hyderabad/',
 'kolkata': 'https://www.carwale.com/used/kolkata/',
 'pune': 'https://www.carwale.com/used/pune/',
 'ahmedabad': 'https://www.carwale.com/used/ahmedabad/',
 'chandigarh': 'https://www.carwale.com/used/chandigarh/',
 'coimbatore': 'https://www.carwale.com/used/coimbatore/',
 'lucknow': 'https://www.carwale.com/used/lucknow/',
 'indore': 'https://www.carwale.com/used/indore/'}

In [8]:
extract_city_links_cardekho = lambda x: (x if 'used-cars+in+' in x and '?' not in x else '')
city_extractor_cardekho = lambda x: x[len('https://www.cardekho.com/used-cars+in+'):]

cardekho_citylinks = extract_citywise_links(
    root_directory='organized_cars_structure/cardekho_data', required_links_extractor=extract_city_links_cardekho, 
    city_extractor=city_extractor_cardekho)

print('Cardekho Available cities and their links \n','-'*30)
cardekho_citylinks

Cardekho Available cities and their links 
 ------------------------------


{'ahmedabad': 'https://www.cardekho.com/used-cars+in+ahmedabad',
 'bangalore': 'https://www.cardekho.com/used-cars+in+bangalore',
 'chennai': 'https://www.cardekho.com/used-cars+in+chennai',
 'delhi-ncr': 'https://www.cardekho.com/used-cars+in+delhi-ncr',
 'hyderabad': 'https://www.cardekho.com/used-cars+in+hyderabad',
 'jaipur': 'https://www.cardekho.com/used-cars+in+jaipur',
 'kolkata': 'https://www.cardekho.com/used-cars+in+kolkata',
 'mumbai': 'https://www.cardekho.com/used-cars+in+mumbai',
 'pune': 'https://www.cardekho.com/used-cars+in+pune'}

In [9]:
extract_city_links_zigwheels = lambda x: x
city_extractor_zigwheels = lambda x: x.split('/')[-1].lower()

zigwheels_citylinks = extract_citywise_links(
    root_directory='organized_cars_structure/zigwheels_data', required_links_extractor=extract_city_links_zigwheels, 
    city_extractor=city_extractor_zigwheels, html_tag='a.zw-cmn-blueColor.zw-new-underline')

print('Zigwheels Available cities and their links \n','-'*30)
zigwheels_citylinks

Zigwheels Available cities and their links 
 ------------------------------


{'delhi': 'https://www.zigwheels.com/used-car/Delhi',
 'mumbai': 'https://www.zigwheels.com/used-car/Mumbai',
 'bangalore': 'https://www.zigwheels.com/used-car/Bangalore',
 'pune': 'https://www.zigwheels.com/used-car/Pune',
 'hyderabad': 'https://www.zigwheels.com/used-car/Hyderabad',
 'chennai': 'https://www.zigwheels.com/used-car/Chennai',
 'kolkata': 'https://www.zigwheels.com/used-car/Kolkata',
 'ahmedabad': 'https://www.zigwheels.com/used-car/Ahmedabad',
 'jaipur': 'https://www.zigwheels.com/used-car/Jaipur',
 'coimbatore': 'https://www.zigwheels.com/used-car/Coimbatore',
 'gurgaon': 'https://www.zigwheels.com/used-car/Gurgaon',
 'noida': 'https://www.zigwheels.com/used-car/Noida',
 'thane': 'https://www.zigwheels.com/used-car/Thane',
 'lucknow': 'https://www.zigwheels.com/used-car/Lucknow',
 'chandigarh': 'https://www.zigwheels.com/used-car/Chandigarh',
 'navi-mumbai': 'https://www.zigwheels.com/used-car/Navi-Mumbai'}

In [10]:
extract_city_links_cartrade = lambda x: 'https://www.cartrade.com'+x if '-' not in  x.split('/')[2] else '//'
city_extractor_cartrade = lambda x: x.split('/')[-2]

cartrade_citylinks = extract_citywise_links(
    root_directory='organized_cars_structure/cartrade_data/', required_links_extractor=extract_city_links_cartrade, 
    city_extractor=city_extractor_cartrade, html_tag='a.font-muli-bold.pt-3.pr-3.pb-3.pl-3.font-13.color-comet.display-block.full-width.full-height.nav-tab-link')

print('Cartrade Available cities and their links \n','-'*30)
cartrade_citylinks = dict(list(cartrade_citylinks.items())[:5])
cartrade_citylinks

Cartrade Available cities and their links 
 ------------------------------


{'delhi': 'https://www.cartrade.com/second-hand/delhi/',
 'mumbai': 'https://www.cartrade.com/second-hand/mumbai/',
 'bangalore': 'https://www.cartrade.com/second-hand/bangalore/',
 'hyderabad': 'https://www.cartrade.com/second-hand/hyderabad/',
 'chennai': 'https://www.cartrade.com/second-hand/chennai/'}

In [11]:
price_range_pairs = {0 : 3, 3 : 10, 10 : 30, 30 : 100}
organizations = ['cars24', 'carwale', 'cardekho', 'zigwheels', 'cartrade']
orgs_cities = [cars24_citylinks, carwale_citylinks, cardekho_citylinks, zigwheels_citylinks, cartrade_citylinks]

filter_links = {
    'cars24' : 
    lambda city, new_min, new_max : f'https://www.cars24.com/buy-used-cars-{city}/?f=listingPrice%3Abw%3A{new_min}%2C{new_max}&sort=bestmatch&serveWarrantyCount=true&listingSource=TabFilter&storeCityId=1',
    
    'carwale' : 
    lambda city, new_min, new_max : f'https://www.carwale.com/used/{city}/?kms=0-&year=0-&budget={new_min}-{new_max}&state=-1&so=-1&sc=-1',
    
    'cardekho' : 
    lambda city, new_min, new_max : f'https://www.cardekho.com/used-cars+{new_min}-lakh-to-{new_max}-lakh+in+{city}',
    
    'zigwheels' : 
    lambda city, new_min, new_max : f'https://www.zigwheels.com/used-car/used-cars-between-{new_min}-to-{new_max}-lakhs-in-{city.capitalize()}',
     
    'cartrade' : 
    lambda city, new_min, new_max : f'https://www.cartrade.com/second-hand/{city}/#so=-1&sc=-1&budget={new_min}-{new_max}'
    }

def extract_filterurls(price_range_pairs, organizations, orgs_city_links, filter_links, output_directory):

    for min, max in price_range_pairs.items():
        for organization, orgwise_links, filter in zip(organizations, orgs_city_links, filter_links.values()):
            filter_name_url = dict()
            os.makedirs('organized_cars_structure/'+organization+'_data/'+output_directory, exist_ok=True)

            for city in orgwise_links.keys():
                if 'cars24' == organization:
                    new_min, new_max = min*100000, max*100000
                else :
                    new_min, new_max = min, max

                filter_url = filter(city, new_min, new_max)
                filtername = f'{city}-{str(min)}-{str(max)}-lakhs_cars.txt'

                filter_name_url[organization+'-'+filtername] = filter_url

            output_file = f"organized_cars_structure/{organization}_data/{output_directory}/filterurls-{str(min)}-{str(max)}-lakhs.txt"
            
            with open(output_file, 'w', encoding='utf-8')as f:
                f.write(str(filter_name_url))

In [12]:
# extract_filterurls(price_range_pairs, organizations, orgs_cities, filter_links, output_directory='cityprice_urls')

In [13]:
def fetch_filtered_data(organizations, org_start_index=0, file_start_index=0, start_index=0,
                          headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}):
    file_num = 0
    iteration = 0
        
    try : 
        for org_id, organization in enumerate(organizations):
            if org_id < org_start_index:
                continue

            import_folder = os.path.join('organized_cars_structure/',organization+'_data/cityprice_urls')
            output_folder = os.path.join('organized_cars_structure/',organization+'_data/cityprice_data')
            filter_urls_file = os.listdir(import_folder)
            for file_num, file in enumerate(filter_urls_file):
                if file_num < file_start_index:
                    continue
                
                import_file = Path(os.path.join(import_folder,file))
                if import_file.is_file():
                    file_content = import_file.read_text()
                    filter_name_url = ast.literal_eval(file_content)
                    for iteration, (filter_name, filter_link) in enumerate(filter_name_url.items()):
                        filter_name = filter_name[len(organization)+1:]
                        if iteration < start_index:
                            continue
                        time.sleep(5)
                        output_file = Path(os.path.join(output_folder, filter_name))
                        os.makedirs(output_folder, exist_ok=True)
                        output_file.touch(exist_ok=True)
                        content = requests.get(filter_link, headers=headers)
                        soup = BeautifulSoup(content.content, 'lxml')
                        output_file.write_text(soup.prettify(), encoding='utf-8')
            print(f'{organization.capitalize()} : Scraped {file_num+file_start_index+1}/{len(filter_urls_file)} files')    
    except Exception as ex:
        print(ex)
        print(f"Execution Stopped at {organizations[org_id+org_start_index]} - File :{filter_urls_file[file_num+file_start_index]}, Link: {iteration}")
    finally:            
        return org_id+org_start_index, file_num+file_start_index, iteration

In [14]:
# org_resume_index, file_resume_index, iteration = fetch_filtered_data(organizations)

In [15]:
# org_resume_index, file_resume_index, iteration = fetch_filtered_data(organizations)

In [16]:
# time.sleep(1800)
# org_resume_index, file_resume_index, iteration = fetch_filtered_data(organizations)

In [17]:
html_tags = {'cars24' : 'a.styles_carCardWrapper__sXLIp',
             'carwale' : 'a.o-C',
             'cardekho' : 'div.titlebox.hover h3.title a',
             'zigwheels' : 'a.fnt-22.zm-cmn-colorBlack.n.zw-sr-headingPadding.clickGtm.saveslotno',
             'cartrade' : 'a.font-muli'}

links_extractor = {'cars24' : lambda x: x,
             'carwale' : lambda x: 'https://www.carwale.com/'+x,
             'cardekho' : lambda x: 'https://www.cardekho.com'+x,
             'zigwheels' : lambda x: x if len(x.split('/'))>5 and '/used-car/' in x else '',
             'cartrade' : lambda x: 'https://www.cartrade.com'+x}

def extract_carurls(file, links_extractor, html_tag='a'):
    with open(file, 'r', encoding='utf-8')as f:
        content = f.read()

    soup = BeautifulSoup(content, 'lxml')
    href_values = [a.get('href') for a in soup.select(html_tag, href=True)]
    links = [links_extractor(link) for link in href_values]
    links = [link for link in links if link.strip() != '']
    return links

def pass_path_to_carurls_extractor(organizations):
    try : 
        for org in organizations:
            root_folder = os.path.join('organized_cars_structure/',org+'_data/cityprice_data/')
            output_folder = os.path.join('organized_cars_structure/',org+'_data/cars_urls/')
            os.makedirs(output_folder, exist_ok=True)
            files = os.listdir(root_folder)
            for file in files:
                import_file = Path(os.path.join(root_folder, file))
                output_file = Path(os.path.join(output_folder, file.replace('_cars','-cars-urls')))
                urls = extract_carurls(import_file, links_extractor[org], html_tag=html_tags[org])
                output_file.touch(exist_ok=True)
                output_file.write_text(str(urls), encoding='utf-8')
            print(f'{org.capitalize()} : Extracted all links \n ')    
    except Exception as ex: 
        print(ex)

In [18]:
# pass_path_to_carurls_extractor(organizations)

In [19]:
extract_file_name = {
    'cars24' : lambda url : url[len('https://www.cars24.com/buy-used-'):-1],
    'carwale': lambda url : url[len('https://www.carwale.com//used/ahmedabad/'):-1].replace('/','-'),
    'cardekho': lambda url, id : url[len('https://www.cardekho.com/used-car-details/used-'):-1].split('_')[0]+str(id+1),
    'zigwheels': lambda url : '-'.join(url[len('https://www.zigwheels.com/used-car/'):-len('-873a-4b89-949f-b12e8998b97e')].split('/')[-2:]),
    'cartrade': lambda url : url[len('https://www.cartrade.com/second-hand/'):-len('/?dc=0')].replace('/', '-')}


def fetch_carsdata(organizations, org_start_index=0, file_start_index=0, url_start_index=0,
                   headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}):
    file_num=0
    url_id=0

    try : 
        for org_id, org in enumerate(organizations):
            if org_id < org_start_index:
                continue
            root_folder = os.path.join('organized_cars_structure/',org+'_data/cars_urls/')
            output_folder = os.path.join('organized_cars_structure/',org+'_data/cars_data/')
            os.makedirs(output_folder, exist_ok=True)
            files = os.listdir(root_folder)
            print(org)
            
            for file_num, file in enumerate(files):
                if file_num < file_start_index:
                    continue
                
                import_file = Path(os.path.join(root_folder, file))
                file_content = import_file.read_text(encoding='utf-8')
                urls = ast.literal_eval(file_content)
            
                for url_id, url in enumerate(urls):
                    if url_id < url_start_index :
                        continue
                    filename_extractor = extract_file_name[org]
                    if org == 'cardekho':
                        output_file = filename_extractor(url, id=url_id)
                    else :
                        output_file = filename_extractor(url)
                        
                    output_path = Path(os.path.join(output_folder, output_file+'.txt'))
                    output_path.touch(exist_ok=True)
                    time.sleep(5)
                    content = requests.get(url=url, headers=headers)
                    soup = BeautifulSoup(content.content, 'lxml')
                    output_path.write_text(soup.prettify(), encoding='utf-8')
            
    except Exception as ex:
        print(ex)
        print(f'Execution stopped at {organizations[org_start_index+org_id]}, File no. : {file_start_index+file_num}, Url no. : {url_start_index+url_id}')
    else :
        print(f'Execution completed - {organizations[org_start_index+org_id]}, Files : {file_start_index+file_num+1}/{len(files)}, Url no. : {url_start_index+url_id}')
    finally :
        return org_start_index+org_id, file_start_index+file_num, url_start_index+url_id

In [20]:
# org_resume_index, file_resume_index, url_resume_index = fetch_carsdata(organizations)

In [21]:
# org_resume_index, file_resume_index, url_resume_index = fetch_carsdata(organizations)

In [22]:
def dict_extractor(file_content, html_tags_keys='a', html_tags_values='a'):
    content = file_content
    soup = BeautifulSoup(content, 'lxml')
    labels = [tag.get_text(strip=True) for tag in soup.select(html_tags_keys)]
    values = [tag.get_text(strip=True) for tag in soup.select(html_tags_values)]
    label_value_pair = {}

    for label, value in zip(labels, values):
        label_value_pair[label] = value
    
    return label_value_pair

In [128]:
def cars24_additional_data_extractor(content, keys=['Displacement (cc)', 'Cylinders', 'Max Power (bhp)', 'Seating Capacity', 'Emission Standard', 'Ground Clearance (mm)', 'Bootspace (litres)']):
    additional_dict = {}
    for key in keys:
        if key not in content :
            additional_dict[key] = None
            continue
        search_key = '\\\"label\\\":\\\"'+key+'\\\",\\\"value\\\":\\\"'
        val = content.split(search_key)[-1].split('\\\"')[0]
        additional_dict[key] = val
    return additional_dict

In [129]:
more_specs = {
    'cars24': cars24_additional_data_extractor,
    'carwale':
            (),
    'cardekho':
            (),
    'zigwheels':
            (),
    'cartrade':
            ()}

In [132]:
html_tags2 = {
    'cars24':
        ('''
        div.styles_itemBody__IMfct p.sc-bhjgvs.bOfVYC, div.styles_itemBody__IMfct p.sc-braxZu.jjIUAi,
        div.styles_priceInfo___5XRj p.sc-bhjgvs.gsgpxI, div.styles_priceInfo___5XRj p.sc-braxZu.dughoY
        ''',
        '''
        div.styles_itemBody__IMfct p.sc-bhjgvs.jhKTHv, div.styles_itemBody__IMfct p.sc-braxZu.kjFjan,
        div.styles_price__3yE9i p.sc-bhjgvs.cmylRs, div.styles_price__3yE9i p.sc-braxZu.hhzsvw'''),
    'carwale':
            (),
    'cardekho':
            (),
    'zigwheels':
            (),
    'cartrade':
            ()}

for org in organizations:
    import_folder = f'organized_cars_structure/{org}_data/cars_data/'
    output_folder = f'organized_cars_structure/{org}_data/cars_dict_data/'
    files = os.listdir(import_folder)
    os.makedirs(output_folder, exist_ok=True)
    for file in files :
        import_file = Path(os.path.join(import_folder, file))
        output_file = Path(os.path.join(output_folder, file.replace('.txt','_dict.txt')))
        print(import_file)
        print(output_file)
        output_file.touch(exist_ok=True)
        if import_file.is_file(): 
            file_content = import_file.read_text(encoding='utf-8')
            dict_data = dict_extractor(file_content,html_tags2[org][0], html_tags2[org][1])
            data_extractor = more_specs[org]
            extra_dict_data = data_extractor(file_content)
            dict_data.update(extra_dict_data)
            print(dict_data)
            # output_file.write_text()
    #     break
    break

organized_cars_structure\cars24_data\cars_data\audi-a5-2017-cars-kolkata-dlr1002486.txt
organized_cars_structure\cars24_data\cars_dict_data\audi-a5-2017-cars-kolkata-dlr1002486_dict.txt
{'Reg. year': 'Jul 2018', 'Fuel': 'Diesel', 'KM driven': '42,000 km', 'Transmission': 'Automatic', 'Engine capacity': '1968cc', 'Ownership': '3rd', 'Make year': 'Jul 2017', 'Spare key': 'Yes', 'Reg number': 'WB02**9299', 'Car price': '₹22.5 lakh', 'Displacement (cc)': '1968', 'Cylinders': '4', 'Max Power (bhp)': '150', 'Seating Capacity': '4', 'Emission Standard': None, 'Ground Clearance (mm)': None, 'Bootspace (litres)': '480'}
organized_cars_structure\cars24_data\cars_data\audi-a6-2013-cars-bangalore-dlr1002102.txt
organized_cars_structure\cars24_data\cars_dict_data\audi-a6-2013-cars-bangalore-dlr1002102_dict.txt
{'Reg. year': 'Feb 2014', 'Fuel': 'Diesel', 'KM driven': '76,178 km', 'Transmission': 'Automatic', 'Engine capacity': '1968cc', 'Ownership': '2nd', 'Make year': 'Feb 2013', 'Spare key': 'No',

KeyboardInterrupt: 

In [133]:
# add service history report and extra charges 